In [1]:
# Для чтения/записи csv файлов
import csv
# Для работы с архивами
import gzip
# Для работы с файловой системой
import os
# Эффективные массивы простых типов
import numpy
# Работа с матрицами (подсчет общих друзей реализован как умножение матрицы графа самое на себя)
import scipy
from scipy.sparse import coo_matrix, csr_matrix

In [5]:
# Пути к данным
dataPath = "./day3"
graphPath = os.path.join(dataPath, "trainGraph")
predictionPath = os.path.join(dataPath,"prediction.gz")

# Основные параметры графа
numUsers = 107182
numLinks = 36192484
maxUserId = 9418031

In [ ]:
# В этих массивах мы будем собирать данные. Инициализируем их заранее нужным размером чтобы 
# небыло лишнего копирования
from_users = numpy.zeros( (numLinks), dtype=numpy.int32 ) 
to = numpy.zeros( (numLinks), dtype=numpy.int32 ) 
data = numpy.ones( (numLinks), dtype=numpy.int32 ) 

# Здесь храним позицию, на которую надо записать новую связь
current = 0

# Итерируемся по файлам в папке
for file in [f for f in os.listdir(graphPath) if f.startswith("part")]:
    csvinput = gzip.open(os.path.join(graphPath, file), mode = 'rt')
    csv_reader = csv.reader(csvinput, delimiter='\t')
    # А теперь по строкам в файле
    for line in csv_reader:
        user = int(line[0]) 
        # Разбираем идшки и маски друзей
        for friendship in line[1].replace("{(", "").replace(")}", "").split("),("):
            parts=friendship.split(",")
            # Записываем связь в массивы и двигаем указатель
            from_users[current] = user
            to[current] = int(parts[0])
            current += 1
            
    # Не забываем закрыть файл
    csvinput.close()    

In [ ]:
# Создаем из массивов матрицу. Изначальна матрица хранится в виде списка [i,j,v], но для эффективной 
# дальнейшей работы нам надо преобразовать в вид [i->[j,v]]
fullMatrix = coo_matrix(
    (mask, (from, to)),
    shape=(maxUserId + 1, maxUserId + 1)).tocsr()

# Массивы больше не нужны, удаляем их из памяти
del from_users
del to
del data

In [5]:
# Считаем транспонированную матрицу (колонки и ряды поменяны местами) и её тоже приводим в вид [i->[j,v]]
reversedMatrix = scipy.transpose(fullMatrix).tocsr()

In [6]:
# Поскольку прогноз нам нужно строить только для части пользователей, остальных из
# исходной матрицы уберем (забьем нулями)
for i in range(maxUserId + 1):
    if i % 11 != 7:
        ptr = fullMatrix.indptr[i]
        ptr_next = fullMatrix.indptr[i+1]
        if ptr != ptr_next:
            fullMatrix.data[ptr:ptr_next].fill(0)
            
# Чтобы нули не мешались при умножении, вычистим их и подуменьшим матрицу
fullMatrix.eliminate_zeros()

In [7]:
# Здесь и происходит основная магия - через умножение матриц получаем счетчики общих друзей,
# По которым сделаем прогноз
commonFriends = fullMatrix.dot(reversedMatrix)

In [8]:
# Теперь осталось его записать в файл. Открываем врайтеры
output = gzip.open(predictionPath, "w")
writer = csv.writer(output, delimiter='\t')

for i in range(maxUserId + 1):
    # Два указателя дают нам границы в которых лежат данные для этого i в матрице
    ptr = commonFriends.indptr[i]
    ptr_next = commonFriends.indptr[i+1] 
    # Если они не равны, значит данные есть и можно экспортировать
    if ptr != ptr_next:
        # Достаем счетчики общих друзей и создаем порядок на них от большего к меньшему
        counts = commonFriends.data[ptr:ptr_next]
        order = numpy.argsort(-counts)
        
        # Не забываем что из прогноза надо убрать себя и своих известных друзей
        mineFriends = set(fullMatrix.indices[fullMatrix.indptr[i]:fullMatrix.indptr[i+1]])
        mineFriends.add(i)
        
        # Достаем идшки друзей, сортируем, фильтруем, обрезаем и пишем
        ids = commonFriends.indices[ptr:ptr_next]        
        writer.writerow([i] + filter(lambda x: x not in mineFriends, ids[order])[:42])      

# Не забываем закрыть файл
output.close()        

In [4]:
def jaccard_score(common_friends_matrix: scipy.sparse.csr_matrix, from_user, to_user) -> float:
    return (common_friends_matrix[from_user][to_user] / 
            (common_friends_matrix[from_user][from_user] + common_friends_matrix[to_user][to_user] - common_friends_matrix[from_user][to_user]))
